# Overview

This notebook is used to generate two decision optimization scenarios for Decision Optimization model `PredictiveMaintenance`. The first one, `Manual` is the manual scenario which takes machine failure predictions from the ML linear regression model and attempts to manually assign, i.e. "fix", maintenance events to days right before predicted failure.  The second scenario, `Optimized`, does not fix any maintenance events and lets the optimization model assign them optimally. As a result of running this notebook, the two scenarios are generated/re-generated and available in the `PredictiveMaintenance` Decision Optimization model.  Once you open the DO model, you will be able to compare the solutions side by side using the DO dashboard.

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM.** <br>
**© Copyright IBM Corp. 2019, 2020 All Rights Reserved.<br> 
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**<br>

## Step 0: If you are running on Cloud Pak for Data as a Service (Public Cloud)

A pre-requisite of running the rest of the cells is to have:
* From the "More" menu above, select the "Insert project token" option
* Enter your "API Key" and "location" (e.g. 'us-south') in the cell below

In [1]:
# If running on Cloud, enter your API key and instance location below.
api_key = ''
location = 'us-south'

## Step 1: Define the ML and DO models

A pre-requisite of running this step is to have:
* Created the Decision Optimization model `PredictiveMaintenance`.  If you have used a different name for your DO model, update the `DO_model` variable below
* Run the AutoAI model and deployed it.  Set the `ML_Deployment_ID` below to the Deployment ID for your deployed model

In [7]:
DO_model="PredictiveMaintenance"
ML_Deployment_ID = ""

## Step 2: Read in the input data needed by the ML and DO models

In [3]:
import os, pandas as pd, requests, urllib3, json, math, numpy as np
from pprint import pprint

# When running on Cloud click the option "Insert project token" - if not running on cloud project is setup below
try:
    project
except NameError:
    # READING AND WRITING PROJECT ASSETS
    import project_lib
    project = project_lib.Project()  

df_parameters = pd.read_csv(project.get_file('parameters.csv'))
df_day = pd.read_csv(project.get_file('day.csv'))
df_machine = pd.read_csv(project.get_file('machine.csv'))
df_planned_production = pd.read_csv(project.get_file('planned_production.csv'))

## Step 3: Obtain failure predictions from the deployed ML model

### Obtain ML Service Endpoint

In [4]:
from ibm_watson_machine_learning import APIClient
import os

if api_key != '':
    wml_credentials = {
        "apikey": api_key,
        "url": 'https://' + location + '.ml.cloud.ibm.com'
    }
else:
    token = os.environ['USER_ACCESS_TOKEN']

    wml_credentials = {
        "token": token,
        "instance_id" : "openshift",
        "url": os.environ['RUNTIME_ENV_APSX_URL'],
        "version": "3.5"
     }

client = APIClient(wml_credentials)

2020-11-03 15:20:57,092 - root - WARNING - Limited tf.compat.v2.summary API due to missing TensorBoard installation.
2020-11-03 15:20:57,146 - root - WARNING - Limited tf.compat.v2.summary API due to missing TensorBoard installation.
2020-11-03 15:20:57,149 - root - WARNING - Limited tf.compat.v2.summary API due to missing TensorBoard installation.
2020-11-03 15:20:57,156 - root - WARNING - Limited tf.compat.v2.summary API due to missing TensorBoard installation.
2020-11-03 15:20:57,195 - root - WARNING - Limited tf.compat.v2.summary API due to missing TensorBoard installation.
2020-11-03 15:20:57,225 - root - WARNING - Limited tf.compat.v2.summary API due to missing TensorBoard installation.
2020-11-03 15:20:57,276 - root - WARNING - Limited tf.summary API due to missing TensorBoard installation.


### Find Deployment Space by name

In [5]:
# Specify a name for the target deployment space
SPACE_NAME = 'IntelligentMaintenance'

# Check is the space already exists
space_uid = ''
for space in client.spaces.get_details()['resources']:
    if space['entity']['name'] == SPACE_NAME:
        space_uid=space['metadata']['id']
        client.set.default_space(space_uid)
        break

if space_uid == '':
    print("Deployment space",SPACE_NAME,"not found.")

### Submit request to WML service and save predictions

_For each machine:_

- Use input data to generate payload in the expected format: 

- Submit a new request to the WML service, obtain the result, and parse the predicted value (day when the machine will fail)

- Generate predicted failure data for surrounding days, using a normal distribution


In [8]:
predictions = {}

for index, row in df_machine.iterrows():
    machine_id = row['id']
    machine_production = df_planned_production[df_planned_production['machine'] == machine_id]
    values = machine_production['production'].values
    values = np.insert(values, 0, row['remaining life'])
    keys = machine_production['day'].values
    keys = np.insert(keys, 0, 'life')
    
    scoring_payload = {
        'input_data': [{
            'fields': keys.tolist(),
            'values': [ values.tolist() ] 
        }]
    }

    response_scoring = client.deployments.score(ML_Deployment_ID, scoring_payload)
    day = response_scoring['predictions'][0]['values'][0][0]
    predictions[machine_id] = int(math.floor(float(day)))

print(predictions)

{'M-01': 10, 'M-02': 8, 'M-03': 8, 'M-04': 6, 'M-05': 7}


In [9]:
cnt = len(df_machine)
data_failure =[]
data_fixedm =[]
nday = len(df_day)

for i in range(cnt):
    machine = df_machine.loc[i,'id']
    life = df_machine.loc[i,'remaining life']

    # Extract the predicted failure day for this machine
    mid = predictions[machine]
    print (str(life) + " --> " + str(mid))
   
    spread = np.random.randint(2, 6)
    n = 1000
    s = np.random.normal(mid, spread/4., n)

    data = [0 for k in range(nday)]
    for k, day in np.ndenumerate(df_day['id']):
        t = 0
        for j in range(1000):
            if round(s[j])==k[0]+1:
                t = t+1   
        data_failure.append((machine, day, round (100.*t/n)))
        if (mid-round(spread/4.)) == k[0]+1:
            data_fixedm.append((machine,day,1))
        else:
            data_fixedm.append((machine,day,0))
            
df_predicted_failure =  pd.DataFrame(data=data_failure, columns=['machine', 'day', 'failure'])  
df_fixed_maintenance = pd.DataFrame(data=data_fixedm, columns=['machine', 'day', 'maintenance'])

14 --> 10
12 --> 8
12 --> 8
9 --> 6
10 --> 7


## Step 4: Generate and solve the Manual and Optimized scenarios

To create the new scenarios, we access template `Scenario 1` in the `PredictiveMaintenance` Decision Optimization model and copy it twice to create two new scenarios. We set the required parameters (data) to specify whether we have fixed maintenance events that should be taken into account (`fix_maintenance = 0` for optimization scenario). We then populate all the input tables in the new scenarios using the data previously loaded/preprocessed). Once we are done, the two new scenarios should be available in the DO model for viewing and comparison.

In [10]:
!pip install --user dd-scenario

from dd_scenario import *

# In order to use the solve() function on Cloud, you must provide an API key when creating the client
if api_key != '':
    client = Client(pc=pc, apikey=api_key)
else:
    client = Client()
    
fw = client.get_model_builder(name=DO_model)
   
for i in range(2):
    if i == 0:
        sc_name = "Manual - NB"
        print("Generating Manual Scenario...")
    else:
        sc_name = "Optimized - NB"
        # set fix_maintenance = 0 to not take into account any manually specified maintenance events
        df_parameters.loc[df_parameters.id == 'fix_maintenance', 'value'] = 0
        df_fixed_maintenance['maintenance'] = 0
        # set maintenance crew size to 1 to use only one crew (instead of default 2)
        df_parameters.loc[df_parameters.id == 'maintenance crew size', 'value'] = 1
        print("Generating Optimized Scenario")
        
    sc = fw.get_scenario(name="Scenario 1")
    # Copy from Scenario 1 to get the right model
    copy = fw.get_scenario(name=sc_name)
    if (copy != None):
        print(" Deleting old...")
        fw.delete_container(copy)    
    copy = sc.copy(sc_name)
    print(" Creating " + sc_name)
    copy.add_table_data("day", df_day, category="input")
    copy.add_table_data("machine", df_machine, category="input")
    copy.add_table_data("predicted_failure", df_predicted_failure, category="input")
    copy.add_table_data("planned_production", df_planned_production, category="input")
    copy.add_table_data("parameters", df_parameters, category="input")
    copy.add_table_data("fixed_maintenance", df_fixed_maintenance, category="input")

    print(" Solving...")
    copy.solve()

print("Done...")

Generating Manual Scenario...
 Deleting old...
 Creating Manual - NB
 Solving...
[2020-11-03T15:32:27Z, SEVERE] InterpreterError : 
  File "model.py", line 248, in <module>
    s = mdl.solve(log_output=True)
  File "model.py", line 4216, in solve
    return self._solve_local(context, force_clean_before_solve, lex_timelimits, lex_mipgaps)
  File "model.py", line 4389, in _solve_local
    get_environment().update_solve_details(details)
  File "environment.py", line 541, in update_solve_details
    self.publish_solve_details(to_publish)
  File "environment.py", line 832, in publish_solve_details
    self.solve_hook.update_solve_details(details)
  File "docloudcontroller.py", line 75, in update_solve_details
    self.channel.send(command.addSolveDetails(False, details))
  File "command.py", line 34, in addSolveDetails
    return json.dumps(command)
  File "__init__.py", line 231, in dumps
    return _default_encoder.encode(obj)
  File "encoder.py", line 199, in encode
    chunks = self.ite